# 1. What I Implemented 

I implemented the following files:
- `example/minimal_example_wil` (minimal example (Mujoco) on the double pendulum xml file)
- `data_collection/real_pend_data.py` (collects data from the real pendulum. Just connect it to Hebi Scope (via the ethernet chord) and it just works)
- `example/minimal_example_real.py` (attempt to run SINDy on double pendulum data. Doesn't work that well)

# 2. `minimal_example_real.py`
## 2.1 Description
I implemented this on top of `example/minimal_example_modular.py`. The basic idea of it is, instead of extracting data from Mujoco, I extract data from the CSV generated by `data_collection/real_pend_data.py`. 

An example of **combined** (i.e., stiched together from multiple trials) output from this `data_collection/real_pend_data.py` can be found on `README_report_25_12_22/outputs/compiled.csv`. 

I then made the time series data *monotonic*, meaning that I stitched together trials and mannually increased the "time" variable (for instance, from the first to second trial, instead of t = 4.999 to 0.000, I changed that to 4.999 to 5.000). See `.../outputs/compiled.csv`. Note SINDy crashes when I don't do this.

## 2.2 Issue
The issue is that the program does not give me reasonable outputs. Usually, the program just fails.Sometimes it gives a partial result like the followign:

<img src="./outputs/trajectory_comparison_e25fcbc4.png" width="400" height="auto" />

I have no idea why this happens, but I have made the following **observations**. 

First the "`_vs`" validation coordinates only have 13 entries and terminate after a second. Furthermore, notice how large the `qvel` and `qacc` values become (note, 0 is the shoulder angular trajectory and 1 is the elbow angular trajectory).

(See `README_.../outputs/irl_vs_test.csv`)
(You can also see the test ("`_v`") trajectory at `README_.../outputs/irl_v_test.csv`)

## 2.3 Trying Different Solutions

 My first instinct was that the problem was with this line in `minimal_example_real.py`:

`simulation_qacc_t = np.gradient(simulation_qvel_t, t, axis=0, edge_order=1)`

The issue could be that, since I a post-computing acceleration instead of collecting it between different trials, the acceleration could be way off. 

Here's a plot of qacc. The spikes correspond to the start of each trial, so this intuition is likely correct.

<img src="./outputs/v_acc_test.png" width="400" height="auto" />


I tried setting `qacc` to 0 at the beginning and end of each trial. I got the following graph, but SINDy still won't work.

<img src="./outputs/v_acc_test_0.png" width="400" height="auto" />

Note that the `qacc` for `vs` looked reasonable in both cases.

<img src="./outputs/vs_acc_test.png" width="400" height="auto" />
<img src="./outputs/vs_acc_test_0.png" width="400" height="auto" />


I tried computing `qacc` in different ways, but that doesn't change qacc. To 
